In [9]:
!pip install ultralytics easyocr opencv-python-headless boto3 pandas

In [11]:
import boto3
import os

s3 = boto3.client('s3')
bucket = 'license-plate-detection-project'

# Create local folders
os.makedirs('input', exist_ok=True)
os.makedirs('model', exist_ok=True)
os.makedirs('output', exist_ok=True)

# Download files
s3.download_file(bucket, 'input/2103099-uhd_3840_2160_30fps.mp4', 'input/video.mp4')
s3.download_file(bucket, 'model/best.pt', 'model/best.pt')


In [14]:
import cv2
import pandas as pd
from ultralytics import YOLO
import easyocr
import os

# Paths
video_path = 'input/video.mp4'
model_path = 'model/best.pt'
output_video_path = 'output/output.mp4'
output_csv_path = 'output/results.csv'

# Load YOLOv8 and OCR
model = YOLO(model_path)
reader = easyocr.Reader(['en'])

# Setup video capture
cap = cv2.VideoCapture(video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))
w, h = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (w, h))

# Get total frames for progress log
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_idx = 0
seen_plates = set()
results_list = []

# Processing loop
while cap.isOpened():
    ret, frame = cap.read()
    if not ret or frame_idx >= total_frames:
        print("End of video or read failure.")
        break

    results = model(frame)[0]
    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
        frame_h, frame_w = frame.shape[:2]

        # Skip boxes near edges = likely partial
        if x1 < 10 or y1 < 10 or x2 > frame_w - 10 or y2 > frame_h - 10:
            continue

        plate_crop = frame[y1:y2, x1:x2]
        ocr_result = reader.readtext(plate_crop)

        for _, text, conf in ocr_result:
            if conf > 0.5:
                plate_text = text.strip().upper()

                # Skip duplicates
                if plate_text in seen_plates:
                    continue
                seen_plates.add(plate_text)

                # Timestamp
                timestamp_sec = frame_idx / fps
                hours = int(timestamp_sec // 3600)
                minutes = int((timestamp_sec % 3600) // 60)
                seconds = timestamp_sec % 60
                timestamp = f"{hours:02}:{minutes:02}:{seconds:05.2f}"

                # Draw box and label
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, plate_text, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)

                # Log result
                results_list.append({
                    'frame': frame_idx,
                    'timestamp': timestamp,
                    'text': plate_text,
                    'confidence': round(conf, 2)
                })

    out.write(frame)
    frame_idx += 1

    if frame_idx % 100 == 0:
        print(f"Processed {frame_idx}/{total_frames} frames")

# Cleanup
cap.release()
out.release()

# Save results
df = pd.DataFrame(results_list)
df.to_csv(output_csv_path, index=False)

print("Detection complete. Output saved.")


[04/24/25 19:44:54] WARNING  Neither CUDA nor MPS are available - defaulting to CPU. Note: This       ]8;id=484395;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/easyocr/easyocr.py\easyocr.py]8;;\:]8;id=717147;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/easyocr/easyocr.py#80\80]8;;\
                             module is much faster with a GPU.                                                     


0: 384x640 3 License_Plates, 1063.3ms
Speed: 3.7ms preprocess, 1063.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 License_Plates, 1386.5ms
Speed: 5.5ms preprocess, 1386.5ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 License_Plates, 1098.7ms
Speed: 3.4ms preprocess, 1098.7ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 License_Plates, 1080.7ms
Speed: 3.5ms preprocess, 1080.7ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 License_Plates, 1053.3ms
Speed: 3.7ms preprocess, 1053.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 License_Plates, 1067.8ms
Speed: 3.4ms preprocess, 1067.8ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 License_Plates, 1097.0ms
Speed: 4.0ms preprocess, 1097.0ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 License_Plates, 1073

In [15]:
s3.upload_file('output/output.mp4', bucket, 'output/output.mp4')
s3.upload_file('output/results.csv', bucket, 'output/results.csv')